In [ ]:
import math
import warnings  # Suppress the UserWarning from StandardScaler
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow
# from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# from scikeras.wrappers import KerasRegressor
# from scikeras.wrappers import KerasClassifier, KerasRegressor
from keras.wrappers.scikit_learn import KerasRegressor

In [ ]:
print(tensorflow.__version__)

In [ ]:
stockPrices = pd.read_csv('yahooScrapping/AAPL.csv', index_col="Date")

In [ ]:
testRatio = 0.2
trainingRatio = 1 - testRatio

trainingSize = int(trainingRatio * len(stockPrices))
testSize = int(testRatio * len(stockPrices))
print(f"Training Size: {trainingSize}")
print(f"Testing Size: {testSize}")

train = stockPrices[:trainingSize][["Close"]]  # Ensure train is a DataFrame
test = stockPrices[trainingSize:][["Close"]].copy()  # Ensure test is a DataFrame

In [ ]:
## Split the time-series data into training seq X and output value Y
def extract_seqX_outcomeY(data, N, offset):
    """
    Split time-series into training sequence X and outcome value Y
    Args:
        data - dataset
        N - window size, e.g., 50 for 50 days of historical stock prices
        offset - position to start the split
    """
    X, y = [], []
    for i in range(offset, len(data)):
        X.append(data[i - N: i])
        y.append(data[i])
    return np.array(X), np.array(y)

In [ ]:
# Calculate the metrics RMSE and MAPE
def calculateRMSE(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    return rmse

def calculateMAPE(y_true, y_pred):
    y_pred, y_true = np.array(y_pred), np.array(y_true)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return mape

In [ ]:
def calculate_perf_metrics(var):
    ### RMSE
    rmse = calculateRMSE(
        np.array(stockPrices[trainingSize:]["Close"]),
        np.array(stockPrices[trainingSize:][var]),
    )
    ### MAPE
    mape = calculateMAPE(
        np.array(stockPrices[trainingSize:]["Close"]),
        np.array(stockPrices[trainingSize:][var]),
    )

    return rmse, mape

In [ ]:
def plot_stock_trend(var, cur_title, stockprices=stockPrices):
    ax = stockprices[["Close", var, "200 Days"]].plot(figsize=(20, 10))
    plt.grid(False)
    plt.title(cur_title)
    plt.axis("tight")
    plt.ylabel("Stock Price ($)")

## Simple Moving Average

In [ ]:
window_size = 50

window_var = f"{window_size}day"

stockPrices[window_var] = stockPrices["Close"].rolling(window_size).mean()

### Include a 200-day SMA for reference
stockPrices["200 Days"] = stockPrices["Close"].rolling(200).mean()

### Plot and performance metrics for SMA model
plot_stock_trend(var=window_var, cur_title="Simple Moving Averages")
rmse_sma, mape_sma = calculate_perf_metrics(var=window_var)

print(f"RMSE for SMA: {rmse_sma}")
print(f"MAPE for SMA: {mape_sma}")

## Exponential Moving Average

In [ ]:
window_ema_var = f"{window_var}_EMA"

# Calculate the 50-day exponentially weighted moving average
stockPrices[window_ema_var] = (
    stockPrices["Close"].ewm(span=window_size, adjust=False).mean()
)
stockPrices["200 Days"] = stockPrices["Close"].rolling(200).mean()

### Plot and performance metrics for EMA model
plot_stock_trend(
    var=window_ema_var, cur_title="Exponential Moving Averages")
rmse_ema, mape_ema = calculate_perf_metrics(var=window_ema_var)

print(f"RMSE for EMA: {rmse_ema}")
print(f"MAPE for EMA: {mape_ema}")

## LSTM Model

In [ ]:
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn')

# Scale our dataset
scaler = StandardScaler()
scaled_data = scaler.fit_transform(stockPrices[["Close"]])
scaled_data_train = scaled_data[:train.shape[0]]

window_size = 50  # define window_size variable
# We use past 50 days’ stock prices for our training to predict the 51th day's closing price.
X_train, y_train = extract_seqX_outcomeY(scaled_data_train, window_size, window_size)

In [ ]:
def create_model(input_shape, layer_units=150, dropout_rate=0.3, optimizer='adam'):
    model = Sequential([
        LSTM(units=layer_units, return_sequences=True, input_shape=input_shape),
        Dropout(dropout_rate),
        LSTM(units=layer_units, return_sequences=True),
        Dropout(dropout_rate),
        LSTM(units=layer_units),
        Dropout(dropout_rate),
        Dense(30, activation="relu"),
        Dense(1)
    ])
    model.compile(loss="mean_squared_error", optimizer=optimizer)
    return model

### Hyperparameter Tuning - Grid Search

In [ ]:
def create_model_wrapper(layer_units=100, dropout_rate=0.3, optimizer='adam'):
    return create_model((X_train.shape[1], 1), layer_units, dropout_rate, optimizer)

model = KerasRegressor(build_fn=create_model_wrapper, verbose=0)

# Define the grid search parameters
param_grid = {
    #'layer_units': [50],
    'dropout_rate': [0.2, 0.3],
    #'optimizer': ['adam'],
    'batch_size': [32],
    'epochs': [30, 50]
}

In [ ]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_result = grid.fit(X_train, y_train)

print("Best parameters found: ", grid_result.best_params_)
print("Best score: ", grid_result.best_score_)

In [ ]:
best_params = grid_result.best_params_
final_model = create_model(
    input_shape=(X_train.shape[1], 1),
    layer_units=50,
    dropout_rate=best_params['dropout_rate'],
    optimizer='adam'
)

In [ ]:
# Callbacks - to prevent overfitting by cutting short epochs and save the best model
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

In [ ]:
history = final_model.fit(
    X_train,
    y_train,
    epochs=best_params['epochs'],
    batch_size=best_params['batch_size'],
    validation_split=0.2,
    verbose=1,
    shuffle=True,
    callbacks=[early_stopping, model_checkpoint]
)

### Predict using Trained LSTM Model

In [ ]:
# Preprocess test data
def preprocess_testdat(data=stockPrices, scaler=scaler, window_size=window_size, test=test):
    raw = data["Close"][len(data) - len(test) - window_size:].values
    raw = raw.reshape(-1, 1)
    raw = scaler.transform(raw)

    X_test = [raw[i - window_size: i, 0] for i in range(window_size, raw.shape[0])]
    X_test = np.array(X_test)

    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    return X_test

X_test = preprocess_testdat()

predicted_price_ = model.predict(X_test)
predicted_price = scaler.inverse_transform(predicted_price_)

# Ensure test DataFrame has a column to store predictions
test["Predictions_lstm"] = predicted_price

In [ ]:
# Debugging output
print(test)
print(test.columns)

In [ ]:
# Evaluate performance
rmse_lstm = calculateRMSE(np.array(test["Close"]), np.array(test["Predictions_lstm"]))
mape_lstm = calculateMAPE(np.array(test["Close"]), np.array(test["Predictions_lstm"]))

print(f"RMSE for LSTM: {rmse_lstm}")
print(f"MAPE for LSTM: {mape_lstm}")

In [ ]:

def plot_stock_trend_lstm(train, test):
    fig, ax = plt.subplots(figsize=(20, 10))
    ax.plot(train.index, train["Close"], label="Train Closing Price")
    ax.plot(test.index, test["Close"], label="Test Closing Price")
    ax.plot(test.index, test["Predictions_lstm"], label="Predicted Closing Price")
    ax.set_title("LSTM Model")
    ax.set_xlabel("Date")
    ax.set_ylabel("Stock Price (USD $)")
    ax.legend()
    plt.show()

plot_stock_trend_lstm(train, test)